In [154]:
import pandas as pd

In [155]:
loans_2007 = pd.read_csv(r'C:\Users\SVR\Documents\LOAN\LoanStats3a.csv', skiprows=1)
half_count = len(loans_2007) / 2
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)
loans_2007 = loans_2007.drop(['desc', 'url'],axis=1)
loans_2007.to_csv(r'C:\Users\SVR\Documents\LOAN\loans_2007.csv', index=False)

C:\Users\SVR\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [156]:
loans_2007 = pd.read_csv(r'C:\Users\SVR\Documents\LOAN\loans_2007.csv',encoding='latin-1')

C:\Users\SVR\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


let us understand each and every feature and determine the final set of features that we can use for our problem.We even need to select our target feature from the pool of features which we feel would be the most appropriate one 

As there are many features,let us analyze by splitting the features into sets of 18.Let us start with first set 18

id: randomly generated field by Lending Club for unique identification purposes only
member_id: also a randomly generated field by Lending Club for unique identification purposes only
funded_amnt: leaks data from the future (after the loan is already started to be funded)
funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
grade: contains redundant information as the interest rate column (int_rate)
sub_grade: also contains redundant information as the interest rate column (int_rate)
emp_title: requires other data and a lot of processing to potentially be useful
issue_d: leaks data from the future (after the loan is already completed funded)
Recall that Lending Club assigns a grade and a sub-grade based on the borrower's interest rate. While the grade and sub_grade values are categorical, the int_rate column contains continuous values, which are better suited for machine learning.



In [157]:
# droping colums of set 1 which are not required

cols_to_drop = ["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"]
loans_2007.drop(cols_to_drop,axis = 1,inplace = True)

second set of features:

zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
out_prncp: leaks data from the future, (after the loan already started to be paid off)
out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)
The out_prncp and out_prncp_inv both describe the outstanding principal amount for a loan, which is the remaining amount the borrower still owes. These 2 columns as well as the total_pymnt column describe properties of the loan after it's fully funded and started to be paid off. This information isn't available to an investor before the loan is fully funded and we don't want to include it in our model.

In [158]:
# droping colums of set 2 which are not required
cols_to_drop_set2 = ["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"]
loans_2007.drop(cols_to_drop_set2,inplace = True,axis = 1)


third set of features:
    
total_rec_int: leaks data from the future, (after the loan already started to be paid off),
total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
recoveries: also leaks data from the future, (after the loan already started to be paid off),
collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).
All of these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower.

In [159]:
#droping colums of set 2 which are not required

cols_to_drop_set3 = ["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"]
loans_2007.drop(cols_to_drop_set3,axis =1,inplace = True)

Deciding on target column:
We should use the loan_status column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. 
Currently, this column contains text values and we need to convert it to a numerical one for training a model

In [160]:
print(loans_2007['loan_status'].value_counts())

Fully Paid                                             33902
Charged Off                                             5658
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Current                                                  201
Late (31-120 days)                                        10
In Grace Period                                            9
Late (16-30 days)                                          5
Default                                                    1
Name: loan_status, dtype: int64


Above tells that there are many categories loan_status column.
From the investor's perspective, we're interested in trying to predict 
which loans will be paid off on time and which ones won't be. 
Only the Fully Paid and Charged Off values describe the final outcome of the loan. 
The other values describe loans that are still on going and where the jury is still out 
on if the borrower will pay back the loan on time or not

Fully Paid - Loan has been fully paid off.
Charged Off	- Loan for which there is no longer a reasonable expectation of further payments.


Since we're interesting in being able to predict which of these 2 values a loan will fall under,
we can treat the problem as a binary classification one.
Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then
transform the Fully Paid values to 1 for the positive case
and the Charged Off values to 0 for the negative case.

In [161]:
#Remove all rows from loans_2007 that contain values other than Fully Paid or Charged Off for the loan_status column
loans_2007 = loans_2007[(loans_2007['loan_status'] == 'Fully Paid') | (loans_2007['loan_status'] == 'Charged Off')]
# above filters rows which matches our condition

In [162]:
# As target column valyes are categorical,will convert into numerical,i.e., converting our problem as binary classification with Fully Paid = 1 and Charged off = 
loan_status = {'loan_status':{'Fully Paid' : 1, 'Charged Off' : 0}}
loans_2007.replace(loan_status,inplace = True)

In [163]:
# lets look at the columns that has only one value,i.e., same value for every loan application.This type of columns are of no use,so,lets remove such columns
drop_columns  = []
for col in loans_2007.columns:
    non_null = loans_2007[col].dropna() # beacuse series unique() method considers pandas nan as 0,so applying this method has chance of returning more than 1 unique value,to avoid that will remove the entries that has NaNs   
    unique_count = len(non_null.unique())
    #print(unique_count)
    if unique_count == 1:
        drop_columns.append(col)
#print(drop_columns)
loans_2007.drop(drop_columns,axis =1,inplace = True)
print(drop_columns)

['initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


## Summary of the things done till now:
we started to become familiar with the columns in the dataset and removed many columns that aren't useful for modeling by taking various things into consideration (columns that have redundant information,colums that has data which requires too much processing to make useful,columns that has information of the future (we don't consider as this kind of information results in undesired model))


Now let us prepare the data for machine learning by focusing on handling missing values, converting categorical columns to numeric columns, and removing any other extraneous columns we encounter throughout this process

In [164]:
null_counts = loans_2007.isnull().sum() #isnull() returns True if value is missing and sum() considers  True as 1,False as 0
print(null_counts)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
pymnt_plan                0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64


In [165]:
#Remove pub_rec_bankruptcies columns as it has many missing values
#removing rows where the columns title,revol_util and last_credit_pull_d has missing values
#loans_2007.drop('pub_rec_bankruptcies',axis =1 , inplace = True)
loans_2007.dropna(axis =0,inplace = True)


In [166]:
#let us check the data types of the columns,text columns need to be converted into numerical values as machine learning models accept only numerical values
print(loans_2007.dtypes.value_counts())

object     12
float64    11
int64       1
dtype: int64


In [167]:
#selecting object columns
object_columns_df = loans_2007.select_dtypes(include=['object'])
print(object_columns_df.head(1))

         term int_rate emp_length home_ownership verification_status  \
0   36 months   10.65%  10+ years           RENT            Verified   

  pymnt_plan      purpose     title addr_state earliest_cr_line revol_util  \
0          n  credit_card  Computer         AZ         Jan-1985      83.7%   

  last_credit_pull_d  
0           Nov-2016  


let us have a look on the text columns

home_ownership: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
verification_status: indicates if income was verified by Lending Club,
emp_length: number of years the borrower was employed upon time of application,
term: number of payments on the loan, either 36 or 60,
addr_state: borrower's state of residence,
purpose: a category provided by the borrower for the loan request,
title: loan title provided the borrower,
int_rate: interest rate of the loan in %,
revol_util: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit

In [168]:
print(len(loans_2007['title'].value_counts()))
print(len(loans_2007['purpose'].value_counts()))

19145
14


In [169]:
#From above,as both more or less same information(just differnt wordings),and as purpose columns contains few discrete values,will conside purpose columns and remove title column
# also let us look for unnique values of other non numeric columns
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for col in cols:
    print(loans_2007[col].value_counts())
#The home_ownership, verification_status, emp_length, and term columns each contain a few discrete categorical values. #
#We should encode these columns as dummy variables and keep them.

RENT        18400
MORTGAGE    17323
OWN          2983
OTHER          96
Name: home_ownership, dtype: int64
Not Verified       16161
Verified           12704
Source Verified     9937
Name: verification_status, dtype: int64
10+ years    8702
< 1 year     4397
2 years      4287
3 years      4013
4 years      3364
5 years      3237
1 year       3135
6 years      2180
7 years      1734
8 years      1447
9 years      1237
n/a          1069
Name: emp_length, dtype: int64
 36 months    28345
 60 months    10457
Name: term, dtype: int64
CA    6988
NY    3701
FL    2804
TX    2668
NJ    1805
IL    1505
PA    1495
GA    1368
VA    1367
MA    1298
OH    1191
MD    1024
AZ     862
WA     811
CO     763
NC     749
CT     737
MI     713
MO     674
MN     604
NV     489
SC     462
OR     439
AL     439
WI     436
LA     426
KY     323
OK     297
KS     267
UT     250
AR     240
DC     208
RI     196
NM     182
WV     176
HI     171
NH     169
DE     111
MT      84
WY      82
AK      78
SD      62
VT  

In [170]:
#now let us work at emp_length column
loans_2007['emp_length']
#We assume that people who may have been working more than 10 years have only really worked for 10 years. We also assume that people who've worked less than a year or if the information is not available that they've worked for 0. This is a general heuristic but it's not perfect.
mapping_dict = {
        "emp_length": {
            "10+ years": 10,
            "9 years": 9,
            "8 years": 8,
            "7 years": 7,
            "6 years": 6,
            "5 years": 5,
            "4 years": 4,
            "3 years": 3,
            "2 years": 2,
            "1 year": 1,
            "< 1 year": 0,
            "n/a": 0
        }
    }
loans_2007.replace(mapping_dict,axis = 1,inplace = True)


C:\Users\SVR\Anaconda3\lib\site-packages\pandas\core\generic.py:3312: UserWarning: the "axis" argument is deprecated and will be removed inv0.13; this argument has no effect
  warn('the "axis" argument is deprecated and will be removed in'


In [171]:
# let us look at addr_state column
print(len(loans_2007['addr_state'].value_counts()))
print(len(loans_2007['last_credit_pull_d'].value_counts()))
print(len(loans_2007['earliest_cr_line'].value_counts()))

49
103
526


In [172]:
#As there are more categories for the columns addr_state,last_credit_pull_d and earliest_cr_line,let us remove them as encoding the into numerical values is not viable
cols = ['last_credit_pull_d', 'addr_state', 'title','earliest_cr_line','pymnt_plan']
loans_2007.drop(cols,axis = 1,inplace = True)

In [173]:
# now its turn to have a look at int_rate column
#loans_2007['revol_util']
# as the values have '%' at the end ,we need to strip that symbol and convert into float type
for i in ['int_rate','revol_util']:
    loans_2007[i]=loans_2007[i].str.rstrip('%')
    loans_2007[i] = loans_2007[i].astype(float)

In [174]:
#Let's now encode the home_ownership, verification_status, title, and term columns as dummy variables so we can use them in our model. We first need to use the Pandas get_dummies method to return a new Dataframe containing a new column for each dummy variable:
# Returns a new Dataframe containing 1 column for each dummy variable.

 
cat_columns = ["home_ownership", "verification_status", "emp_length", "purpose", "term"]
dummy_df = pd.get_dummies(loans_2007[cat_columns])
loans = pd.concat([loans_2007, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)

In [175]:
#now we are almost done with the cleaning up of the data.Now let us see how many predictor variables are there in the final data on which we apply our machine learning model

print(len(loans.columns))
#so,out of 38 columns,7 are predictor variables and one is target variable.The algorithm will make predictions about whether or not a loan will be paid off on time, which is contained in the loan_status column of the dataset.
#if we have a look at the data,clearly the data is imbalanced,as There are about 6 times as many loans that were paid off on time (positive case, label of 1) than those that weren't (negative case, label of 0)
#Imbalances can cause issues with many machine learning algorithms, where they appear to have high accuracy, but actually aren't learning from the training data



37


ERROR METRIC
Before diving in and selecting an algorithm to apply to the data, we should select an error metric.
An error metric will help us figure out when our model is performing well, and when it's performing poorly.
let's say we're using a machine learning model to predict whether or not we should fund a loan on the Lending Club platform. Our objective in this is to make money -- we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid off. An error metric will help us determine if our algorithm will make us money or lose us money.

FASLE POSITIVES AND FALSE NEGATIVES
In this case, we're primarily concerned with false positives and false negatives. 
Both of these are different types of misclassifications.
With a false positive, we predict that a loan will be paid off on time, but it actually isn't. This costs us money, since we fund loans that lose us money. With a false negative, we predict that a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money, since we didn't fund a loan that actually would have been paid off.

WHICH ONE TO TAKE CARE OF?
Since we're viewing this problem from the standpoint of a conservative investor, we need to treat false positives differently than false negatives. A conservative investor would want to minimize risk, and avoid false positives as much as possible. They'd be more okay with missing out on opportunities (false negatives) than they would be with funding a risky loan (false positives).




We need to optimize for
1.high recall (true positive rate(TPR))
2.low fall-out (false positive rate(FPR))

1.TPR
True positive rate is the number of true positives divided by the number of true positives plus the number of false negatives. This divides all the cases where we thought a loan would be paid off and it was by all the loans that were paid off:
tpr = tp / (tp + fn)

2.FPR
False positive rate is the number of false positives divided by the number of false positives plus the number of true negatives. This divides all the cases where we thought a loan would be paid off but it wasn't by all the loans that weren't paid off:
fpr = fp / (fp + tn)


False Positive Rate -- "what percentage of my 1 predictions are incorrect?"
In this case, "what percentage of the loans that I fund would not be repaid?"

True Positive Rate -- "what percentage of all the possible 1 predictions am I making?"
In this case, "what percentage of loans that could be funded would I fund?"

Generally, if we want to reduce false positive rate, true positive rate will also go down. This is because if we want to reduce the risk of false positives, we wouldn't think about funding riskier loans in the first place.

In [230]:
#TIME TO USE MACHINE LEARNING MODEL TO MODEL OUR DATA
#A good first algorithm to apply to binary classification problems is logistic regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
cols = loans.columns
train_cols = cols.drop("loan_status")
features = loans[train_cols]
target = loans["loan_status"]
lr.fit(features, target)
predictions = lr.predict(features)


CROSS VALIDATION:

-While we generated predictions in the last screen, those predictions were overfit. They were overfit because we generated predictions using the same data that we trained our model on. When we use this to evaluate error, we get an unrealistically high depiction of how accurate the algorithm is, because it already "knows" the correct answers.

-In order to get a realistic depiction of the accuracy of the algorithm, we'll need to use cross validation to generate predictions. Cross validation splits the dataset into groups, then makes predictions on each group using the other groups as training data. This ensures that we don't overfit by generating predictions on the same data that we train our algorithm with.

-We can perform cross validation using the cross_val_predict method of scikit-learn. cross_val_predict allows us to pass in a classifier, the features, and the target.

-We'll create an instance of KFold, which will perform 3 fold cross validation across our dataset. We set random_state to 1 to ensure that the folds are always consistent, and we can compare scores between runs. If we don't, each fold will be randomized every time, making it hard to tell if we're improving our model or not.

-If we pass the instance of KFold into cross_val_predict, it will then perform 3 fold cross validation to generate unbiased predictions.

-Once we have cross validated predictions, we can compute true positive rate and false positive rate.

In [231]:
from sklearn.cross_validation import cross_val_predict, KFold
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

In [232]:

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.9991828087167071
0.9990864242645715


In [233]:
print('predicted value counts : \n %s '%(predictions.value_counts()))
print("----------------------------------------------------------------")
print('actual value counts : \n %s '%(loans['loan_status'].value_counts()))

predicted value counts : 
 1    38770
0       32
dtype: int64 
----------------------------------------------------------------
actual value counts : 
 1    33281
0     5521
Name: loan_status, dtype: int64 


From above,we can see that there are many 0's classified as 1's.This is because,the dataset is imbalanced as there are more number of samples classified as class 1 than class 0.

Two methods to overcome this proble
1.oversampling and undersampling
2.Tell the classifier to penalize misclassifications of the less prevalent class more than the other class

1.oversampling and undersampling
oversampling-Copy rows multiple times. One way to equalize the 0s and 1s is to copy rows where loan_status is 0
undersampling- we could do this by deleting rows where loan_status is 1.

-Unfortunately, none of these techniques are especially easy. The second method we mentioned earlier, telling the classifier to penalize certain rows more, is actually much easier to implement using scikit-learn

-We can do this by setting the class_weight parameter to balanced when creating the LogisticRegression instance. 

-This tells scikit-learn to penalize the misclassification of the minority class during the training process. 

-The penalty means that the logistic regression classifier pays more attention to correctly classifying rows where loan_status is 0. 

-This lowers accuracy when loan_status is 1, but raises accuracy when loan_status is 0.

-By setting the class_weight parameter to balanced, the penalty is set to be inversely proportional to the class frequencies.

-This would mean that for the classifier, correctly classifying a row where loan_status is 0 is 6 times more important than correctly classifying a row where loan_status is 1.


## MANUAL PENALTIES
We significantly improved false positive rate in the last screen by balancing the classes, which reduced true positive rate. 
Our true positive rate is now around 67%, and our false positive rate is around 40%. From a conservative investor's standpoint, it's reassuring that the false positive rate is lower because it means that we'll be able to do a better job at avoiding bad loans than if we funded everything.
However, we'd only ever decide to fund 67% of the total loans (true positive rate), so we'd immediately reject a good amount of loans

We can try to lower the false positive rate further by assigning a harsher penalty for misclassifying the negative class. 
While setting class_weight to balanced will automatically set a penalty based on the number of 1s and 0s in the column, we can also set a manual penalty.
In the last screen, the penalty scikit-learn imposed for misclassifying a 0 would have been around 5.89 (since there are 5.89 times as many 1s as 0s).

We can also specify a penalty manually if we want to adjust the rates more. To do this, we need to pass in a dictionary of penalty values to the class_weight parameter:

penalty = {
    0: 10,
    1: 1
}

In [238]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_predict
lr = LogisticRegression(class_weight="balanced")
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)


0.6267554479418886
0.6144710396491869


In [239]:
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight=penalty)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)


0.225635593220339
0.22071989767951763


observations from the above output:
1.when we set penalty to balanced the tpr and fpr values are 0.6267554479418886
0.6144710396491869 respectively

2.Next we tried to impose more penalty on misclassification of  class 0 using values defined by dictionary penalty.

3.Both tpr and fpr values are around 22%.It means,we have reduced fpr but at the same time tpr values are also decreased which is not desired

4.While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. Given that we're approaching this as a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs.

5.While we could tweak the penalties further, it's best to move to trying a different model right now, for larger potential false positive rate gains

6.Let's try a more complex algorithm, random forest. Random forests are able to work with nonlinear data, and learn complex conditionals. Logistic regressions are only able to work with linear data. Training a random forest algorithm may enable us to get more accuracy due to columns that correlate nonlinearly with loan_status.
    

In [240]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict
rf = RandomForestClassifier(class_weight="balanced", random_state=1)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)
predictions = pd.Series(predictions)

In [241]:
# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.9707627118644068
0.9700347158779463


Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.
Ultimately, our best model had a false positive rate of 7%, and a true positive rate of 20%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 7% of borrowers defaulting, and that the pool of 20% of borrowers is large enough to make enough interest money to offset the losses.

If we had randomly picked loans to fund, borrowers would have defaulted on 14.5% of them, and our model is better than that, although we're excluding more loans than a random strategy would. Given this, there's still quite a bit of room to improve:

We can tweak the penalties further.
We can try models other than a random forest and logistic regression.
We can use some of the columns we discarded to generate better features.
We can ensemble multiple models to get more accurate predictions.
We can tune the parameters of the algorithm to achieve higher performance.